# Clase 10 - Evaluación de Clustering

KNN: K-Nearest Neighbors o K vecinos más cercanos

## Carga de Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10, 10]
np.random.seed(42)

from sklearn.datasets import fetch_20newsgroups_vectorized, fetch_20newsgroups

## Ejercicio

In [2]:
news_20 = fetch_20newsgroups()
datos = fetch_20newsgroups_vectorized()
noticias = datos.data
noticias

<11314x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [4]:
#CLASIFICACIONES QUE TIENE
datos.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

VEMOS QUE EL DATASET TIENE 20 CLUSTERS NATURALES (20 PAGINAS DE NOTICIAS DISTINTAS)

In [6]:
from sklearn.cluster import MiniBatchKMeans

#NO LE IMPORTA LAS CLASIFICACIONES A PRIORI
estimador = MiniBatchKMeans(n_clusters=20)
#SE LLENA CON LOS DATOS DE LA TABLA
estimador.fit(noticias)
#SE OBTIENE LA CLASIFICICACION DE CLUSTER CADA CADA ELEMENTO
etiquetas_pred = estimador.labels_
etiquetas_pred

array([5, 5, 4, ..., 0, 3, 2])

In [7]:
#MEDIDAS DE EVALUACION EXTERNA
from sklearn.metrics import homogeneity_completeness_v_measure, adjusted_rand_score

clases = datos.target
clases

array([17,  7, 10, ..., 14, 12, 11])

In [8]:
#La función homogeneity_completeness_v_measure nos devuelve una tupla con la homogeneidad, la completación y la medida v
#SE IMPRIMEN LAS MEDIDAS EXTERNAS, HOMOGENEIDAD, COMPLETEZ, LA MEDIDA V
#SE DAN LOS VALORES REALES Y SE DAN LOS VALORES PREDICHOS
print(homogeneity_completeness_v_measure(clases, etiquetas_pred))

(0.05701504495109323, 0.07904323547167809, 0.0662459294574962)


**METODO DE CLUSTER NO ESTA HECHO PARA HACER PREDICCIONES NO ESPERES BUENOS RESULTADOS DE PREDICCION**

In [9]:
#Vemos que estos clusters tienen mas completación que homogeneidad (es decir, los clusters son menos homogéneos que la homogeneidad de la distribución de las clases en clusters.

#El v-measure tiene un rango de (0, 1) con lo cual estos clusters no son muy buenos.

#Un problema que tiene el criterio de homogeneidad y completación es que no son invariantes del orden de las clases.

print(homogeneity_completeness_v_measure(etiquetas_pred, clases))

(0.07904323547167807, 0.05701504495109322, 0.0662459294574962)


In [10]:
#Para datasets de menos de 1000 observaciones o para un número de clusters mayor que 10, se recomienda usar el Indice de Rand ajustado, adjusted_rand_score.
#PARA CLUSTERS DE MAS DE 10 SE USA LA MEDIDA AJUSTADA
print(adjusted_rand_score(clases, etiquetas_pred))
print(adjusted_rand_score(etiquetas_pred, clases))

0.020046031128384836
0.020046031128384836


La métrica ARI tiene un rango de (-1,1) asi que pese a que este grupo de clusters no es el peor, no es muy bueno

Podemos usar medidas de evaluación externa para hacer validación cruzada como haríamos con cualquier problema de regresión/clasificación.

Métricas definidas en cross_val_score incluyen:

- **adjusted_rand_score**
- **completeness_score**
- **homogeneity_score**
- **v_measure_score**

In [11]:
#SE PUEDE HACER LA VALIDACION CRUZADA PERO LA LOGICA ES LA MISMA, CASI NUNCA SE OBTIENEN BUENOS RESULTADOS
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(X=noticias, y=clases, estimator=MiniBatchKMeans(), 
                             scoring="adjusted_rand_score", cv=5)
print("Datos de validacion de cruzada:")
print(resultados.mean())

Datos de validacion de cruzada:
0.019443826922611006


In [12]:
#Medidas de evaluación interna¶

#Dichas medidas se usan cuando las clases verdaderas no se conocen de antemano

from sklearn.metrics import silhouette_score

#Estas medidas internas se usan en sklearn pasando dos argumentos, el dataset de entrenamiento y las etiquetas de los clusters.

#En primer lugar tenemos el Coeficiente de Silueta (silhouette_score)

print("Coeficiente de silueta")
print(silhouette_score(noticias, etiquetas_pred))

#SE HACE UN FOR PARA VER COMO CAMBIA EL COEFICIENTE DE SILUETA ANTE DIFERENTES VALORES DE K(CLUSTER)
for k in [5,10,20,30]:
    estimador = MiniBatchKMeans(n_clusters=k)
    estimador.fit(noticias)
    etiquetas_pred = estimador.labels_
    print(f"{(k,silhouette_score(noticias, etiquetas_pred))}")

Coeficiente de silueta
-0.07104590850763973
(5, 0.01324346252702793)
(10, -0.017728164451369273)
(20, -0.07934842059413448)
(30, -0.035259804549091014)
